<div >
<img src = "figs/ans_banner_1920x200.png" />
</div>

# Caso-taller: Reconocimiento facial con K-medias y K-medoides

En este caso-taller vamos a aplicar K-medias y K-medoides para agrupar rostros de indiviudos. Utilizaremos la base de datos conocida como Caras Olivetti. Esta base contiene diez imágenes diferentes de  40 personas distintas. Para algunas personas, las imágenes se tomaron en diferentes momentos, variando la iluminación, las expresiones faciales (ojos abiertos/cerrados, sonriendo/sin sonreír) y los detalles faciales (gafas/sin gafas). Todas las imágenes fueron tomadas contra un fondo homogéneo oscuro con los sujetos en una posición frontal erguida (con tolerancia a algún movimiento lateral). Esta base viene incluida dentro de la libraría `scikit-learn` y fueron provistas por T&T Laboratories Cambridge para que la comunidad académica la utilice para la evaluación de modelos predictivos. 

## Instrucciones generales

1. Para desarrollar el *cuaderno* primero debe descargarlo.

2. Para responder cada inciso deberá utilizar el espacio debidamente especificado.

3. La actividad será calificada sólo si sube el *cuaderno* de jupyter notebook con extensión `.ipynb` en la actividad designada como "entrega calificada por el personal".

4. El archivo entregado debe poder ser ejecutado localmente por el tutor. Sea cuidadoso con la especificación de la ubicación de los archivos de soporte, guarde la carpeta de datos en el mismo `path` de su cuaderno, por ejemplo: `data`.

## Desarrollo


### 1.Carga de datos 

Los datos pueden ser bajados directamente utilizando la librería `scikit-learn`,  la función que le permite hacer esto es `fetch_olivetti_faces`. 

In [ ]:
# Utilice este espacio para escribir el código.
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_olivetti_faces
import matplotlib.pyplot as plt

: 

In [ ]:
olivetti = fetch_olivetti_faces(data_home = 'data')

En primer lugar, se importan algunos de los paquetes y funciones que serán utilizados en adelante. A continuación, se implementa la función `fetch_olivetti_faces` para cargar el conjunto de datos "Olivetti Faces". Esta base de datos contiene imágenes de caras de personas. El argumento data_home='data' especifica la carpeta donde se descargará el conjunto de datos (si no existe, se creará). La función devuelve un diccionario que contiene imágenes de caras y otra información relevante.

### 2.  Análisis preliminar 

#### 2.1. Describa los contenidos de la base de datos, tenga en cuenta que debe detallar los elementos, la dimensión de la base, y la dimensión de las imágenes. 

El atributo olivetti.DESCR contiene una descripción del conjunto de datos "Olivetti Faces". Cuando se accede a olivetti.DESCR, obtienes información sobre el conjunto de datos, incluyendo detalles sobre las imágenes de las caras y cómo se recopilaron. Esta descripción proporciona una visión general del conjunto de datos para que los usuarios puedan comprender su contenido y contexto.

El conjunto de datos "Olivetti Faces" consiste en imágenes de caras tomadas entre abril de 1992 y abril de 1994 en AT&T Laboratories Cambridge. Este conjunto de datos se utiliza para la tarea de reconocimiento de caras. A continuación, un resumen de la información proporcionada en el resultado de olivetti.DESCR:

**Descripción General:**

- El conjunto de datos contiene un conjunto de imágenes de caras tomadas en AT&T Laboratories Cambridge.
- Las imágenes pertenecen a 40 sujetos distintos.
- Cada sujeto tiene diez imágenes diferentes, lo que permite considerar variaciones en la iluminación, expresiones faciales (ojos abiertos/cerrados, sonrisa/no sonrisa) y detalles faciales (gafas/sin gafas).
- Las imágenes se tomaron con los sujetos en posición vertical y frontal, con alguna tolerancia para el movimiento lateral.

**Características del Conjunto de Datos:**

- Clases: 40 (cada clase representa a un sujeto diferente).
- Total de muestras: 400 (10 imágenes por sujeto).
- Dimensionalidad: 4096 (cada imagen se representa como un vector de características de longitud 4096).
- Características: Valores reales, en el rango entre 0 y 1.

**Formato de las Imágenes:**

- Las imágenes se cuantizan a 256 niveles de gris y se almacenan como enteros sin signo de 8 bits.
- El cargador convertirá estas imágenes en valores de punto flotante en el intervalo [0, 1], lo cual es más conveniente para muchos algoritmos.

**Objetivo del Conjunto de Datos:**

- El "target" o objetivo de este conjunto de datos es un entero entre 0 y 39 que indica la identidad de la persona en la imagen.
- Sin embargo, debido a que hay solo 10 ejemplos por clase (sujeto), este conjunto de datos es más interesante para enfoques no supervisados o semi-supervisados.

**Tamaño de las Imágenes:**

- El conjunto de datos original contenía imágenes de tamaño 92x112 píxeles.
- La versión disponible aquí contiene imágenes de tamaño 64x64 píxeles.

In [ ]:
olivetti.DESCR

El atributo olivetti.images contiene una matriz que representa las imágenes de caras en el conjunto de datos "Olivetti Faces". Cada fila de esta matriz representa una imagen, y las dimensiones de las imágenes se han convertido en un arreglo unidimensional de valores.

In [ ]:
olivetti.images

A continuación, se obtiene la tupla (n_samples, h, w) con los valores de las dimensiones de la matriz de imágenes en el conjunto de datos. Los significados de los valores son los siguientes:

n_samples: El número total de muestras en el conjunto de datos, es decir, la cantidad de imágenes.
h: La altura de cada imagen (cantidad de filas de píxeles).
w: El ancho de cada imagen (cantidad de columnas de píxeles).

Así, se evidencia que hay un total de 400 imágenes, con 64 píxeles de alto y 64 de ancho cada una de ellas.

In [ ]:
n_samples, h, w = olivetti.images.shape

In [ ]:
n_samples

In [ ]:
h

In [ ]:
w

El atributo olivetti.target en el conjunto de datos "Olivetti Faces" contiene un arreglo que representa las etiquetas o identificadores de las personas correspondientes a cada imagen de cara en el conjunto de datos. Cada valor en este arreglo indica a qué persona pertenece la imagen correspondiente

In [ ]:
olivetti.target

El atributo olivetti.data en el conjunto de datos "Olivetti Faces" contiene una matriz que representa las imágenes de caras preprocesadas como un arreglo bidimensional. Cada fila de esta matriz corresponde a una imagen, y cada columna representa una característica o píxel.

In [ ]:
olivetti.data

Después, se crea una variable X que contiene la matriz de datos de imágenes preprocesadas del conjunto de datos "Olivetti Faces". Se convierte la matriz X en un DataFrame de pandas donde cada fila de este DataFrame representa una imagen, y cada columna corresponde a un píxel o característica (con un total de 64x64 columnas).

Se crea una variable y que contiene las etiquetas de las personas correspondientes a cada imagen en el conjunto de datos y se convierte el arreglo de etiquetas y en un DataFrame de pandas con una sola columna llamada "pid". Cada fila de este DataFrame contendrá la etiqueta correspondiente a la persona en la imagen.

Se crea un nuevo DataFrame df al combinar el DataFrame y (con las etiquetas) y el DataFrame X (con los datos de las imágenes) utilizando la función join(). Esto crea un DataFrame que tiene las etiquetas y los datos de las imágenes juntos.

df.head(): Muestra las primeras filas del DataFrame df, lo que da una vista previa de cómo se estructuran los datos.

In [ ]:
X = olivetti.data
X = pd.DataFrame(X)

y = olivetti.target
y = pd.DataFrame(y, columns=['pid'])
n_samples, h, w = olivetti.images.shape
df = y.join(X)
df.head()

Para obtener estadísticas descriptivas del DataFrame df, se usa el método describe() proporcionado por pandas. Este método proporciona un resumen de las estadísticas básicas para cada columna numérica en el DataFrame.

In [ ]:
df.describe()

#### 2.2. Genere una figura con una imágen de cada uno de los individuos

Primero, se generó una visualización para todos los individuos. El siguiente código crea una cuadrícula de subgráficas para mostrar imágenes de caras contenidas en el DataFrame faces_tplot. Cada subgráfica contiene una imagen de cara y se ajusta a la disposición definida por n_row y n_col. Las imágenes se obtienen del DataFrame y se muestran en una cuadrícula utilizando la biblioteca matplotlib.

In [ ]:
faces_tplot=df.drop(columns=['pid'])
n_row=20
n_col=20

plt.figure(figsize=(1.5 * n_col, 2.2 * n_row))
plt.subplots_adjust(0.6, 0.5, 1.5, 1.5)
for i in range(n_row * n_col):
    plt.subplot(n_row, n_col, i + 1)
    plt.imshow(faces_tplot.iloc[i].to_numpy().reshape((h, w)), cmap=plt.cm.gray)
    plt.xticks(())
    plt.yticks(())
plt.tight_layout()
plt.show()

El código usado previamente es modificado para que solo muestre la primera imagen de cada uno de los individuos, dando como resultado un total de 20 imágenes.

In [ ]:
n_row = 20
n_col = 20 

plt.figure(figsize=(2.5 * n_col, 2.2 * n_row))
plt.subplots_adjust(0.6, 0.5, 1.5, 1.5)

for i in range(n_row):
    plt.subplot(n_row, n_col, i + 1)
    image_index = i * n_col    
    plt.imshow(faces_tplot.iloc[image_index].to_numpy().reshape((h, w)), cmap=plt.cm.gray)   
    plt.xticks(())
    plt.yticks(())

plt.tight_layout()
plt.show()

#### 2.3. Transforme las imágenes en un data frame donde cada columna contiene los valores estandarizados del nivel de gris en la imagen y una columna con la etiqueta de cada imagen. 

El siguiente código calcula la estandarización de los datos de las imágenes en el DataFrame X y luego concatena estas características estandarizadas con las etiquetas del DataFrame y en un nuevo DataFrame df. Las características estandarizadas permiten trabajar con datos con media cero y desviación estándar uno

In [ ]:
mu = X.mean()
sigma = X.std()
Z = (X - mu)/sigma

In [ ]:
df = pd.concat([y, Z], ignore_index=True, axis=1)

In [ ]:
df = df.rename(columns={0: 'pid'})

In [ ]:
df.head()

### 3. Clasificando de forma no supervisada

En esta sección trataremos de clasificar las imágenes de los individuos de forma no supervisada. Para ello utilizaremos los algoritmos de k-medias y k-medoides. Dado que sabemos que el dataset consta de 40 personas diferentes, utilizaremos esta información para pedirle a los algoritmos que encuentren k=40 clusters y examinaremos su precisión.


#### 3.1. Implemente k-medias sobre los datos estandarizados en el punto anterior. Al implementar utilice  `random_state=123` y `n_init=10`. (Esto puede tomar mucho tiempo y requerir mucho RAM, puede aprovechar los recursos de [Google Colab](https://colab.research.google.com/))

In [ ]:
from sklearn.cluster import KMeans

kmeans_40 = KMeans(n_clusters = 40, random_state = 123, n_init=10).fit_predict(X)

Este código aplica el algoritmo de clustering K-Means al conjunto de datos de imágenes de caras en el DataFrame X y asigna cada imagen a uno de los 40 clusters. El resultado es un arreglo kmeans_40 que contiene la asignación de cluster para cada imagen en el conjunto de datos.

#### 3.2. Evalúe la precisión del algoritmo para agrupar las imágenes de los individuos. 

In [ ]:
import seaborn as sns


data = {'y': olivetti.target, 'cluster': kmeans_40}
comparison_df = pd.DataFrame(data)
cross_tab = pd.crosstab(comparison_df['y'], comparison_df['cluster'])
plt.figure(figsize=(10, 8))
sns.heatmap(cross_tab, annot=True, cmap="YlGnBu")
plt.show()

#### 3.3. Usando PCA reduzca la dimensión de la matriz de predictores. Retenga el numero de componentes que explican el 95% de la varianza y vuelva a utilizar k-medias para clasificar las imágenes comentado si la precisión mejoró. 

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95, random_state=123)
X_pca = pca.fit_transform(X)
kmeans_40_pca = KMeans(n_clusters=40, random_state=123, n_init=10).fit_predict(X_pca)
df_pca = pd.DataFrame({'pid': y['pid'], 'cluster': kmeans_40_pca})

In [ ]:
comparison_df = pd.DataFrame(df_pca)
cross_tab = pd.crosstab(comparison_df['pid'], comparison_df['cluster'])
plt.figure(figsize=(10, 8))
sns.heatmap(cross_tab, annot=True, cmap="YlGnBu")
plt.show()

(Utilice este espacio para describir el procedimiento, análisis y conclusiones)

#### 3.4. Utilice ahora el algoritmo por  K-medoides (use el mismo random state a los puntos anteriores), comente si mejoró la precisión total y para cada grupo de imágenes.

In [27]:
# Utilice este espacio para escribir el código.

(Utilice este espacio para describir el procedimiento, análisis y conclusiones)

### 4. Número de clusters óptimo

En el punto anteriores utilizamos nuestro conocimiento previo sobre los datos para elegir el número de clusters. En este punto, verifique si 40 es realmente el número de cluster que usted elegiría según los criterios estudiados en los cuadernos teóricos. Discuta los resultados obtenidos.

In [28]:
# Utilice este espacio para escribir el código.

(Utilice este espacio para describir el procedimiento, análisis y conclusiones)